Tem que continuar pesquisando essas colunas com valores constantes

In [2]:
import pandas as pd
from scipy.io import arff
data = arff.loadarff('arritmias.txt')
df = pd.DataFrame(data[0])

In [3]:
# Primeiro, vamos separar as colunas numéricas
cols_nums = df.select_dtypes(include=['float'])
total_numericos = cols_nums.shape[1]
cols_nums.shape

(452, 272)

In [4]:
nums = df.select_dtypes(include=['float'])

In [5]:
nums.describe()

,Age,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,T,...,Amp_V6_1,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10
count,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,452.000000,444.000000,...,452.000000,452.000000,452.000000,452.000000,452.000000,452.0,452.000000,452.000000,452.000000,452.000000
mean,46.471239,166.188053,68.170354,88.920354,155.152655,367.207965,169.949115,90.004425,33.676991,36.150901,...,-0.302434,-0.278982,9.048009,-1.457301,0.003982,0.0,0.514823,1.222345,19.326106,29.473230
std,16.466631,37.170340,16.590803,15.364394,44.842283,33.385421,35.633072,25.826643,45.431434,57.858255,...,0.603551,0.548876,3.472862,2.002430,0.050118,0.0,0.347531,1.426052,13.503922,18.493927
min,0.000000,105.000000,6.000000,55.000000,0.000000,232.000000,108.000000,0.000000,-172.000000,-177.000000,...,-5.600000,-4.100000,0.000000,-28.600000,0.000000,0.0,-0.800000,-6.000000,-44.200000,-38.600000
25%,36.000000,160.000000,59.000000,80.000000,142.000000,350.000000,148.000000,79.000000,3.750000,14.000000,...,-0.500000,-0.425000,6.600000,-2.100000,0.000000,0.0,0.400000,0.500000,11.450000,17.550000
50%,47.000000,164.000000,68.000000,86.000000,157.000000,367.000000,162.000000,91.000000,40.000000,41.000000,...,-0.200000,0.000000,8.800000,-1.100000,0.000000,0.0,0.500000,1.350000,18.100000,27.900000
75%,58.000000,170.000000,79.000000,94.000000,175.000000,384.000000,179.000000,102.000000,66.000000,63.250000,...,0.000000,0.000000,11.200000,0.000000,0.000000,0.0,0.700000,2.100000,25.825000,41.125000
max,83.000000,780.000000,176.000000,188.000000,524.000000,509.000000,381.000000,205.000000,169.000000,179.000000,...,2.700000,0.000000,23.600000,0.000000,0.800000,0.0,2.400000,6.000000,88.800000,115.900000


Atributos categóricos

In [6]:
cats = df.select_dtypes(include=['O'])

In [7]:
cats.describe()

,Sex,Existence_ragged_R_wave,Existence_diphasic_derivation_R_wave,Existence_ragged_P_wave,Existence_diphasic_drivation_P_wave,Existence_ragged_T_wave,Existence_diphasic_derivation_T_wave,Class
count,452,452,452,452,452,452,452,452
unique,2,2,2,2,2,2,2,13
top,b'1',b'0',b'0',b'0',b'0',b'0',b'0',b'1'
freq,249,451,447,447,450,450,448,245


In [8]:
bytes_columns = df.select_dtypes(include=['O']).columns

for column in df[bytes_columns]:
    if column == "Sex":
        df[column] = df[column].apply(lambda sex: 1 if sex == b'0' else 0)
    else: 
        df[column] = df[column].apply(lambda x: int(x))


**Missing Values**

Vamos começar identificando quantos missing values temos e em quais atributos se encontram

In [9]:
df.isnull().sum()[lambda x: x != 0]

T               8
P              22
QRST            1
J             376
Heart_rate      1
dtype: int64

In [10]:
# Agora vamos ver quantos % relativo ao total está faltando
TOTAL = df.shape[0]
missing = df.isnull().sum()[lambda x: x != 0]
missing.apply(lambda x: (x/TOTAL)*100)

T              1.769912
P              4.867257
QRST           0.221239
J             83.185841
Heart_rate     0.221239
dtype: float64

80% da coluna "J" são "Missing Values", Vamos removê-la

In [11]:
df.drop('J',axis=1,inplace=True)

Vamos remover também as linhas das colunas que possuem apenas um valor missing

In [12]:
df = df.dropna(subset=["QRST","Heart_rate"])

Para as colunas "T" e "P" iremos usar o knn imputer da scikit learn

In [13]:
from sklearn.impute import KNNImputer

In [14]:
imputer = KNNImputer()

In [15]:
df[['T', 'P']] = imputer.fit_transform(df[['T','P']].to_numpy())

In [16]:
df.isnull().sum()[lambda x: x!=0]

Series([], dtype: int64)

Lidamos com todos os missing values

**Outliers**

Vamos utilizar LocalOutlierFactor, um algoritmo não supervisionado que mede a densidade do conjunto de amostras e compara com o dado, caso um dado se encontre distante de uma área de densidade, pode ser um Outlier

In [17]:
from sklearn.neighbors import LocalOutlierFactor

Vamos criar uma função que itera por cada atributo desse DataFrame, buscando Outliers

In [18]:
def train_model(column):
    clf = LocalOutlierFactor(n_neighbors=int(nums[column].shape[0]*0.99)) # Numero de vizinhos
    # Pelo que eu entendi do algoritmo e da documentação, iremos marcar os que estiverem diferentes de 99%
    y_pred = clf.fit_predict(nums[column].values.reshape(-1,1))

    return y_pred

In [19]:
def find_outliers(l):
    positions = []
    for i, item in enumerate(l):
        if item == -1:
            positions.append(i)
            
    return positions

In [20]:
nums = df.select_dtypes(include=['float'])
count = 0 # Para evitar cobrir muito espaço em pdf
for i in nums.columns:
    # Primeiro treinamos para aquela coluna
    print(i)
    tmp_train = train_model(i)
    
    # Depois salvamos quais são os valores que destoaram do modelo
    tmp_list_out = find_outliers(tmp_train)
    print(tmp_list_out)
    
    if count == 20: break
    
    count += 1

Age
[]
Height
[140, 314]
Weight
[212]
QRS_duration
[]
P-R
[391]
Q-T
[]
T_interval
[27]
P_interval
[]
QRS
[]
T
[]
P
[269]
QRST
[]
Heart_rate
[314]
Q_wave
[75, 101]
R_wave
[]
S_wave
[]
R_lin_wave
[115, 192, 212, 420]
S_lin_wave
[]
Number_itrinsic_deflections
[84]
DII_Q_wave
[29, 189]
DII_R_wave
[]


In [21]:
pd.set_option("display.max_columns", None)

Vamos analisar os casos para "Height"

In [22]:
nums.iloc[[140,314]]

,Age,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,T,P,QRST,Heart_rate,Q_wave,R_wave,S_wave,R_lin_wave,S_lin_wave,Number_itrinsic_deflections,DII_Q_wave,DII_R_wave,DII_S_wave,DII_R_lin_wave,DII_S_lin__wave,DII_Number_itrinsic_deflections,DII_Existence_ragged_R_wave,DII_Existence_diphasic_derivation_R_wave,DII_Existence_ragged_P_wave,DII_Existence_diphasic_drivation_P_wave,DII_Existence_ragged_T_wave,DII_Existence_diphasic_derivation_T_wave,DIII_Q_wave,DIII_R_wave,DIII_S_wave,DIII_R_lin__wave,DIII_S_lin__wave,DIII_Number_itrinsic_deflections,DIII_Existence_ragged_R_wave,DIII_Existence_diphasic_derivation_R_wave,DIII_Existence_ragged_P_wave,DIII_Existence_diphasic_drivation_P_wave,DIII_Existence_ragged_T_wave,DIII_Existence_diphasic_derivation_T_wave,AVR_1,AVR_2,AVR_3,AVR_4,AVR_5,AVR_6,AVR_7,AVR_8,AVR_9,AVR_10,AVR_11,AVR_12,AVL_1,AVL_2,AVL_3,AVL_4,AVL_5,AVL_6,AVL_7,AVL_8,AVL_9,AVL_10,AVL_11,AVL_12,AVF_1,AVF_2,AVF_3,AVF_4,AVF_5,AVF_6,AVF_7,AVF_8,AVF_9,AVF_10,AVF_11,AVF_12,V1_1,V1_2,V1_3,V1_4,V1_5,V1_6,V1_7,V1_8,V1_9,V1_10,V1_11,V1_12,V2_1,V2_2,V2_3,V2_4,V2_5,V2_6,V2_7,V2_8,V2_9,V2_10,V2_11,V2_12,V3_1,V3_2,V3_3,V3_4,V3_5,V3_6,V3_7,V3_8,V3_9,V3_10,V3_11,V3_12,V4_1,V4_2,V4_3,V4_4,V4_5,V4_6,V4_7,V4_8,V4_9,V4_10,V4_11,V4_12,V5_1,V5_2,V5_3,V5_4,V5_5,V5_6,V5_7,V5_8,V5_9,V5_10,V5_11,V5_12,V6_1,V6_2,V6_3,V6_4,V6_5,V6_6,V6_7,V6_8,V6_9,V6_10,V6_11,V6_12,Amp_JJ_wave,Amp_Q_wave,Amp_R_wave,Amp_S_wave,Amp_R_lin_wave,Amp_S_lin_wave,Amp_P_wave,Amp_T_wave,QRSA,QRSTA,Amp_DII_1,Amp_DII_2,Amp_DII_3,Amp_DII_4,Amp_DII_5,Amp_DII_6,Amp_DII_7,Amp_DII_8,Amp_DII_9,Amp_DII_10,Amp_DIII_1,Amp_DIII_2,Amp_DIII_3,Amp_DIII_4,Amp_DIII_5,Amp_DIII_6,Amp_DIII_7,Amp_DIII_8,Amp_DIII_9,Amp_DIII_10,Amp_AVR_1,Amp_AVR_2,Amp_AVR_3,Amp_AVR_4,Amp_AVR_5,Amp_AVR_6,Amp_AVR_7,Amp_AVR_8,Amp_AVR_9,Amp_AVR_10,Amp_AVL_1,Amp_AVL_2,Amp_AVL_3,Amp_AVL_4,Amp_AVL_5,Amp_AVL_6,Amp_AVL_7,Amp_AVL_8,Amp_AVL_9,Amp_AVL_10,Amp_AVF_1,Amp_AVF_2,Amp_AVF_3,Amp_AVF_4,Amp_AVF_5,Amp_AVF_6,Amp_AVF_7,Amp_AVF_8,Amp_AVF_9,Amp_AVF_10,Amp_V1_1,Amp_V1_2,Amp_V1_3,Amp_V1_4,Amp_V1_5,Amp_V1_6,Amp_V1_7,Amp_V1_8,Amp_V1_9,Amp_V1_10,Amp_V2_1,Amp_V2_2,Amp_V2_3,Amp_V2_4,Amp_V2_5,Amp_V2_6,Amp_V2_7,Amp_V2_8,Amp_V2_9,Amp_V2_10,Amp_V3_1,Amp_V3_2,Amp_V3_3,Amp_V3_4,Amp_V3_5,Amp_V3_6,Amp_V3_7,Amp_V3_8,Amp_V3_9,Amp_V3_10,Amp_V4_1,Amp_V4_2,Amp_V4_3,Amp_V4_4,Amp_V4_5,Amp_V4_6,Amp_V4_7,Amp_V4_8,Amp_V4_9,Amp_V4_10,Amp_V5_1,Amp_V5_2,Amp_V5_3,Amp_V5_4,Amp_V5_5,Amp_V5_6,Amp_V5_7,Amp_V5_8,Amp_V5_9,Amp_V5_10,Amp_V6_1,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10
141,1.0,780.0,6.0,85.0,165.0,237.0,150.0,106.0,88.0,30.0,30.0,52.0,138.0,0.0,28.0,28.0,0.0,0.0,16.0,0.0,32.0,12.0,20.0,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,20.0,24.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,32.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,1.0,0.0,24.0,20.0,24.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,16.0,24.0,24.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,32.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,32.0,32.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,28.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,32.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,64.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,32.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,12.2,-11.7,0.0,0.0,1.5,1.1,0.7,10.6,0.0,0.0,3.9,-0.7,3.2,-1.2,1.7,1.7,8.3,30.0,-0.7,0.0,2.7,-12.3,14.9,0.0,-0.1,-0.8,8.7,-1.5,-0.4,-6.0,4.2,0.0,0.0,0.0,-1.6,-1.0,-5.3,-17.9,0.7,-1.4,12.3,-13.3,0.0,0.0,0.8,-0.7,-5.2,-12.2,-0.4,0.0,3.0,-6.2,9.1,-0.9,1.0,1.0,9.2,20.6,-1.0,-2.8,7.5,0.0,0.0,0.0,-1.9,-0.9,7.0,-0.5,0.0,-0.4,6.4,-1.6,0.0,0.0,0.8,-0.5,7.3,3.6,0.7,0.0,19.1,-13.5,0.0,0.0,2.0,1.2,19.3,29.3,1.8,0.0,20.2,-11.6,0.0,0.0,1.4,1.7,21.9,32.4,0.1,0.0,17.2,-7.6,0.0,0.0,1.2,0.7,13.5,16.1,0.5,0.0,5.0,-4.6,0.0,0.0,1.3,0.7,2.7,5.5
316,0.0,608.0,10.0,83.0,126.0,232.0,128.0,60.0,125.0,21.0,-50.0,102.0,163.0,0.0,20.0,48.0,0.0,0.0,12.0,0.0,40.0,28.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,32.0,40.0,0.0,0.0,0.0,44.0,0.0,0.0,0.0,0.0,0.0

Certamente são dados errados(Note que o 316, que é 314 na lista mostrada, causou diversos outliers em outros atributos)

In [23]:
height = df[df["Height"] > 200]

In [24]:
height

,Age,Sex,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,T,P,QRST,Heart_rate,Q_wave,R_wave,S_wave,R_lin_wave,S_lin_wave,Number_itrinsic_deflections,Existence_ragged_R_wave,Existence_diphasic_derivation_R_wave,Existence_ragged_P_wave,Existence_diphasic_drivation_P_wave,Existence_ragged_T_wave,Existence_diphasic_derivation_T_wave,DII_Q_wave,DII_R_wave,DII_S_wave,DII_R_lin_wave,DII_S_lin__wave,DII_Number_itrinsic_deflections,DII_Existence_ragged_R_wave,DII_Existence_diphasic_derivation_R_wave,DII_Existence_ragged_P_wave,DII_Existence_diphasic_drivation_P_wave,DII_Existence_ragged_T_wave,DII_Existence_diphasic_derivation_T_wave,DIII_Q_wave,DIII_R_wave,DIII_S_wave,DIII_R_lin__wave,DIII_S_lin__wave,DIII_Number_itrinsic_deflections,DIII_Existence_ragged_R_wave,DIII_Existence_diphasic_derivation_R_wave,DIII_Existence_ragged_P_wave,DIII_Existence_diphasic_drivation_P_wave,DIII_Existence_ragged_T_wave,DIII_Existence_diphasic_derivation_T_wave,AVR_1,AVR_2,AVR_3,AVR_4,AVR_5,AVR_6,AVR_7,AVR_8,AVR_9,AVR_10,AVR_11,AVR_12,AVL_1,AVL_2,AVL_3,AVL_4,AVL_5,AVL_6,AVL_7,AVL_8,AVL_9,AVL_10,AVL_11,AVL_12,AVF_1,AVF_2,AVF_3,AVF_4,AVF_5,AVF_6,AVF_7,AVF_8,AVF_9,AVF_10,AVF_11,AVF_12,V1_1,V1_2,V1_3,V1_4,V1_5,V1_6,V1_7,V1_8,V1_9,V1_10,V1_11,V1_12,V2_1,V2_2,V2_3,V2_4,V2_5,V2_6,V2_7,V2_8,V2_9,V2_10,V2_11,V2_12,V3_1,V3_2,V3_3,V3_4,V3_5,V3_6,V3_7,V3_8,V3_9,V3_10,V3_11,V3_12,V4_1,V4_2,V4_3,V4_4,V4_5,V4_6,V4_7,V4_8,V4_9,V4_10,V4_11,V4_12,V5_1,V5_2,V5_3,V5_4,V5_5,V5_6,V5_7,V5_8,V5_9,V5_10,V5_11,V5_12,V6_1,V6_2,V6_3,V6_4,V6_5,V6_6,V6_7,V6_8,V6_9,V6_10,V6_11,V6_12,Amp_JJ_wave,Amp_Q_wave,Amp_R_wave,Amp_S_wave,Amp_R_lin_wave,Amp_S_lin_wave,Amp_P_wave,Amp_T_wave,QRSA,QRSTA,Amp_DII_1,Amp_DII_2,Amp_DII_3,Amp_DII_4,Amp_DII_5,Amp_DII_6,Amp_DII_7,Amp_DII_8,Amp_DII_9,Amp_DII_10,Amp_DIII_1,Amp_DIII_2,Amp_DIII_3,Amp_DIII_4,Amp_DIII_5,Amp_DIII_6,Amp_DIII_7,Amp_DIII_8,Amp_DIII_9,Amp_DIII_10,Amp_AVR_1,Amp_AVR_2,Amp_AVR_3,Amp_AVR_4,Amp_AVR_5,Amp_AVR_6,Amp_AVR_7,Amp_AVR_8,Amp_AVR_9,Amp_AVR_10,Amp_AVL_1,Amp_AVL_2,Amp_AVL_3,Amp_AVL_4,Amp_AVL_5,Amp_AVL_6,Amp_AVL_7,Amp_AVL_8,Amp_AVL_9,Amp_AVL_10,Amp_AVF_1,Amp_AVF_2,Amp_AVF_3,Amp_AVF_4,Amp_AVF_5,Amp_AVF_6,Amp_AVF_7,Amp_AVF_8,Amp_AVF_9,Amp_AVF_10,Amp_V1_1,Amp_V1_2,Amp_V1_3,Amp_V1_4,Amp_V1_5,Amp_V1_6,Amp_V1_7,Amp_V1_8,Amp_V1_9,Amp_V1_10,Amp_V2_1,Amp_V2_2,Amp_V2_3,Amp_V2_4,Amp_V2_5,Amp_V2_6,Amp_V2_7,Amp_V2_8,Amp_V2_9,Amp_V2_10,Amp_V3_1,Amp_V3_2,Amp_V3_3,Amp_V3_4,Amp_V3_5,Amp_V3_6,Amp_V3_7,Amp_V3_8,Amp_V3_9,Amp_V3_10,Amp_V4_1,Amp_V4_2,Amp_V4_3,Amp_V4_4,Amp_V4_5,Amp_V4_6,Amp_V4_7,Amp_V4_8,Amp_V4_9,Amp_V4_10,Amp_V5_1,Amp_V5_2,Amp_V5_3,Amp_V5_4,Amp_V5_5,Amp_V5_6,Amp_V5_7,Amp_V5_8,Amp_V5_9,Amp_V5_10,Amp_V6_1,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10,Class
141,1.0,0,780.0,6.0,85.0,165.0,237.0,150.0,106.0,88.0,30.0,30.0,52.0,138.0,0.0,28.0,28.0,0.0,0.0,16.0,0,0,0,0,0,0,0.0,32.0,12.0,20.0,12.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,20.0,24.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,1.0,40.0,32.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,1.0,0.0,24.0,20.0,24.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,28.0,16.0,24.0,24.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,36.0,32.0,0.0,0.0,0.0,48.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,32.0,32.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,28.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,32.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.0,64.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,32.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.7,0.0,12.2,-11.7,0.0,0.0,1.5,1.1,0.7,10.6,0.0,0.0,3.9,-0.7,3.2,-1.2,1.7,1.7,8.3,30.0,-0.7,0.0,2.7,-12.3,14.9,0.0,-0.1,-0.8,8.7,-1.5,-0.4,-6.0,4.2,0.0,0.0,0.0,-1.6,-1.0,-5.3,-17.9,0.7,-1.4,12.3,-13.3,0.0,0.0,0.8,-0.7,-5.2,-12.2,-0.4,0.0,3.0,-6.2,9.1,-0.9,1.0,1.0,9.2,20.6,-1.0,-2.8,7.5,0.0,0.0,0.0,-1.9,-0.9,7.0,-0.5,0.0,-0.4,6.4,-1.6,0.0,0.0,0.8,-0.5,7.3,3.6,0.7,0.0,19.1,-13.5,0.0,0.0,2.0,1.2,19.3,29.3,1.8,0.0,20.2,-11.6,0.0,0.0,1.4,1.7,21.9,32.4,0.1,0.0,17.2,-7.6,0.0,0.0,1.2,0.7,13.5,16.1,0.5,0.0,5.0,-4.6,0.0,0.0,1.3,0.7,2.7,5.5,5
316,0.0,1,608.0,10.0,83.0,126.0,232.0,128.

In [25]:
df = df.drop([141, 316])

Vamos dividir em Treino e teste. 
E testar como um modelo de árvore de decisão irá performar 

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

train, teste = train_test_split(df, test_size=0.1,random_state=42)

train_X = train.drop("Class", axis=1)
train_y = train.Class

teste_X = teste.drop("Class", axis=1)
test_y = teste.Class

forest_model = RandomForestClassifier(random_state=1)
forest_model.fit(train_X, train_y)
preds = forest_model.predict(teste_X)
print(accuracy_score(test_y, preds))
print(f1_score(test_y, preds, average='weighted'))

0.7555555555555555
0.711776522284997


Aplicando a normalização e testando nosso modelo 

In [52]:
train, teste = train_test_split(df, test_size=0.1, random_state=42)

train_X = train.drop("Class", axis=1)
train_y = train.Class

teste_X = teste.drop("Class", axis=1)
test_y = teste.Class

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
normalized_x_train = pd.DataFrame(
    scaler.fit_transform(train_X),
    columns = train_X.columns
)

forest_model = RandomForestClassifier(random_state=1)
forest_model.fit(normalized_x_train, train_y)

normalized_x_test = pd.DataFrame(
    scaler.transform(teste_X),
    columns = train_X.columns
)

preds = forest_model.predict(normalized_x_test)
print(accuracy_score(test_y, preds))
print(f1_score(test_y, preds, average='weighted'))

0.7555555555555555
0.711776522284997


### 2 - Seleção de variáveis

In [53]:
# TODO: Concertar o problema que a professora apontou nesta célula
"""
from sklearn.feature_selection import VarianceThreshold
selector = VarianceThreshold()
VT_data = pd.DataFrame(selector.fit_transform(df), columns=selector.get_feature_names_out())
VT_data
"""

'\nfrom sklearn.feature_selection import VarianceThreshold\nselector = VarianceThreshold()\nVT_data = pd.DataFrame(selector.fit_transform(df), columns=selector.get_feature_names_out())\nVT_data\n'

In [54]:
"""
from sklearn.ensemble import RandomForestRegressor

# Separar os dados em features (X) e target (y)
X = train.drop('Class', axis=1)
y = train['Class']

# Criar o modelo Random Forest
rf = RandomForestRegressor()

# Treinar o modelo
rf.fit(X, y)

# Obter a importância das variáveis
feature_importance = rf.feature_importances_

# Criar um DataFrame com as variáveis e suas importâncias
importance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importance})

# Ordenar as variáveis por importância
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df
"""

"\nfrom sklearn.ensemble import RandomForestRegressor\n\n# Separar os dados em features (X) e target (y)\nX = train.drop('Class', axis=1)\ny = train['Class']\n\n# Criar o modelo Random Forest\nrf = RandomForestRegressor()\n\n# Treinar o modelo\nrf.fit(X, y)\n\n# Obter a importância das variáveis\nfeature_importance = rf.feature_importances_\n\n# Criar um DataFrame com as variáveis e suas importâncias\nimportance_df = pd.DataFrame({'Variable': X.columns, 'Importance': feature_importance})\n\n# Ordenar as variáveis por importância\nimportance_df = importance_df.sort_values(by='Importance', ascending=False)\nimportance_df\n"

In [55]:
from sklearn.ensemble import RandomForestRegressor

# Separar os dados em features (X) e target (y)
# X_train2select = train.drop('Class', axis=1)
# y_train2select = train['Class']

# Criar o modelo Random Forest
rf_train = RandomForestRegressor()

# Treinar o modelo
rf_train.fit(train_X, train_y)

# Obter a importância das variáveis
feature_importance = rf_train.feature_importances_

# Criar um DataFrame com as variáveis e suas importâncias
importance_df = pd.DataFrame({'Variable': train_X.columns, 'Importance': feature_importance})

# Ordenar as variáveis por importância
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

,Variable,Importance
4,QRS_duration,0.080251
222,Amp_V1_5,0.078135
13,Heart_rate,0.063820
89,V1_4,0.032236
8,P_interval,0.032193
...,...,...
120,V3_11,0.000000
126,V4_5,0.000000
203,Amp_AVL_6,0.000000
128,V4_7,0.000000


In [41]:
bla = list(importance_df.head()['Variable'])

In [42]:
df[bla]

,QRS_duration,Amp_V1_5,Heart_rate,P_interval,V1_4
0,91.0,0.0,63.0,121.0,0.0
1,81.0,0.0,53.0,39.0,0.0
2,138.0,6.5,75.0,102.0,76.0
3,100.0,0.0,71.0,143.0,0.0
5,100.0,0.0,84.0,91.0,0.0
...,...,...,...,...,...
447,80.0,0.0,63.0,117.0,0.0
448,100.0,0.0,73.0,73.0,0.0
449,108.0,12.6,84.0,116.0,44.0
450,93.0,0.0,80.0,63.0,0.0


In [ ]:
def selecting_best_mutualinfo():
    
    tmp_gain_df = gain_df[gain_df['Gain']>0]
    columns2select = [20, 30, 40, 50, 100, 150, tmp_gain_df.shape[0]]
    
    for qtd in columns2select:
        atribute_columns = list(tmp_gain_df.head(qtd)['Variable'])
        tmp_df = train_X[atribute_columns]
        
        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(tmp_df, train_y)

        preds_tmp = forest_model.predict(test_X[atribute_columns])


        print()
        print('Número de atributos selecionados: ', qtd)
        print('Acurácia: ', end='')
        print(accuracy_score(test_y, preds_tmp))
        print('F1:       ', end='')
        print(f1_score(test_y, preds_tmp, average='weighted'))
        print()
        
selecting_best_mutualinfo()

In [43]:
select_num = [20, 30, 50, 100, 150, 200, 250, 272]

for num in select_num:
    atribute_columns = list(importance_df.head(num)['Variable'])
    tmp_df = train_X[atribute_columns]
    print(tmp_df.shape)
    print(train_y.shape)
    forest_model = RandomForestClassifier(random_state=3)
    
    forest_model.fit(tmp_df, train_y)
    
    preds_tmp = forest_model.predict(teste_X[atribute_columns])
    
    
    print(accuracy_score(test_y, preds_tmp))
    print(f1_score(test_y, preds_tmp, average='weighted'))
    print()
    

(403, 20)
(403,)
0.7111111111111111
0.6226455026455026

(403, 30)
(403,)
0.7111111111111111
0.6264004096262161

(403, 50)
(403,)
0.7333333333333333
0.6642796426402983

(403, 100)
(403,)
0.7555555555555555
0.6900834005918752

(403, 150)
(403,)
0.7333333333333333
0.6886772486772486

(403, 200)
(403,)
0.7555555555555555
0.7183068783068782

(403, 250)
(403,)
0.7777777777777778
0.7405291005291005

(403, 272)
(403,)
0.7333333333333333
0.6659259259259258



In [65]:

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
"""
selector = SelectKBest(f_classif, k=20)
k_best_data = pd.DataFrame(selector.fit_transform(X, y), columns=selector.get_feature_names_out())
k_best_data
"""

'\nselector = SelectKBest(f_classif, k=20)\nk_best_data = pd.DataFrame(selector.fit_transform(X, y), columns=selector.get_feature_names_out())\nk_best_data\n'

In [54]:
df.columns[df.nunique() <= 1]

Index(['S_lin_wave', 'AVL_5', 'AVL_7', 'AVL_12', 'AVF_9', 'V4_9', 'V4_10',
       'V5_5', 'V5_7', 'V5_9', 'V5_11', 'V6_5', 'V6_10', 'V6_11',
       'Amp_S_lin_wave', 'Amp_AVL_6', 'Amp_V5_6', 'Amp_V6_6'],
      dtype='object')

In [78]:
df.loc[:, (df == 0).all()].columns

Index(['S_lin_wave', 'AVL_5', 'AVL_7', 'AVL_12', 'AVF_9', 'V4_9', 'V4_10',
       'V5_5', 'V5_7', 'V5_9', 'V5_11', 'V6_5', 'V6_10', 'V6_11',
       'Amp_S_lin_wave', 'Amp_AVL_6', 'Amp_V5_6', 'Amp_V6_6'],
      dtype='object')

In [80]:
df['DII_Existence_ragged_P_wave'].value_counts()

0.0    447
1.0      1
Name: DII_Existence_ragged_P_wave, dtype: int64

In [81]:
# Possible contants 
# [33,53,67,178]
def bla():
    df_no_constants = df.drop(df.columns[df.nunique() <= 1], axis=1)
    return df_no_constants.iloc[:, 33]
print(bla())

0      0.0
1      0.0
2      0.0
3      0.0
5      0.0
      ... 
447    0.0
448    0.0
449    0.0
450    0.0
451    0.0
Name: DII_Existence_ragged_P_wave, Length: 448, dtype: float64


In [82]:
def bla():
    df_no_constants = df.drop(df.columns[df.nunique() <= 1], axis=1)
    return df_no_constants.iloc[:, 53]
print(bla())

0      0.0
1      0.0
2      0.0
3      0.0
5      0.0
      ... 
447    0.0
448    0.0
449    0.0
450    0.0
451    0.0
Name: AVR_5, Length: 448, dtype: float64


In [85]:
def bla():
    df_no_constants = df.drop(df.columns[df.nunique() <= 1], axis=1)
    return df_no_constants.iloc[:, 67]
print(bla())

0      0.0
1      0.0
2      0.0
3      0.0
5      0.0
      ... 
447    0.0
448    0.0
449    0.0
450    0.0
451    0.0
Name: AVL_9, Length: 448, dtype: float64


In [87]:
def bla():
    df_no_constants = df.drop(df.columns[df.nunique() <= 1], axis=1)
    return df_no_constants.iloc[:, 178]
print(bla())

0      0.0
1      0.0
2      0.0
3      0.0
5      0.0
      ... 
447    0.0
448    0.0
449    0.0
450    0.0
451    0.0
Name: Amp_AVR_6, Length: 448, dtype: float64


In [83]:
df['DII_Existence_ragged_P_wave'].value_counts()

0.0    447
1.0      1
Name: DII_Existence_ragged_P_wave, dtype: int64

In [84]:
df['AVR_5'].value_counts()

0.0     447
32.0      1
Name: AVR_5, dtype: int64

In [86]:
df['AVL_9'].value_counts()

0.0    447
1.0      1
Name: AVL_9, dtype: int64

In [88]:
df['Amp_AVR_6'].value_counts()

 0.0    447
-0.4      1
Name: Amp_AVR_6, dtype: int64

In [60]:
# Percebemos que existem colunas com valores constantes
# graças ao warning do k-means, vale a pena colocar no Relatório
df['S_lin_wave'].value_counts()

0.0    448
Name: S_lin_wave, dtype: int64

Vamos ver se retirando as colunas constantes melhora o algoritmo

In [89]:
# Vou ter que ver isso melhor

def func2(x_train, y_train, x_test, y_test):
    hash_table = {}
    columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
    for value in columns2select:
        selector = SelectKBest(f_classif, k=value)
        k_best_data = pd.DataFrame(selector.fit_transform(x_train, y_train), columns=selector.get_feature_names_out())
        #k_best_data

        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(k_best_data, y_train)

        preds_tmp = forest_model.predict(x_test[selector.get_feature_names_out()])

        hash_table[value] = [accuracy_score(y_test, preds_tmp), f1_score(y_test, preds_tmp, average='weighted')]
        #print('Para as ', value, 'primeiras colunas')
        #print(accuracy_score(test_y, preds_tmp))
        #print(f1_score(test_y, preds_tmp, average='weighted'))
        #print()
        #print('-'*30)
    return hash_table

def evitando_global():
    df_no_constants = df.drop(df.columns[df.nunique() <= 1], axis=1)
    df_no_constants = df.drop(['DII_Existence_ragged_P_wave','AVR_5','AVL_9','Amp_AVR_6'], axis=1)
    train, teste = train_test_split(df_no_constants, test_size=0.1,random_state=42)

    train_X = train.drop("Class", axis=1)
    train_y = train.Class

    teste_X = teste.drop("Class", axis=1)
    test_y = teste.Class
    
    no_const = func2(train_X, train_y, teste_X, test_y)
    print(no_const)

evitando_global()

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 18  64  66  70  79 127 128 135 137 139 141 147 152 153 160 199 259 269] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 18  64  66  70  79 127 128 135 137 139 141 147 152 153 160 199 259 269] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/kalai/anaconda3/lib/python3.9/site-packa

{20: [0.6888888888888889, 0.5958597883597883], 30: [0.7111111111111111, 0.6334259259259258], 40: [0.7111111111111111, 0.6380406212664277], 50: [0.7555555555555555, 0.7077248677248676], 60: [0.7555555555555555, 0.711776522284997], 100: [0.7777777777777778, 0.7478781244298487], 150: [0.7555555555555555, 0.7057507155867812], 200: [0.7111111111111111, 0.642057420418076], 250: [0.7555555555555555, 0.7142267325873883]}


In [66]:
def func():
    hash_table = {}
    columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
    for value in columns2select:
        selector = SelectKBest(f_classif, k=value)
        k_best_data = pd.DataFrame(selector.fit_transform(train_X, train_y), columns=selector.get_feature_names_out())
        #k_best_data

        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(k_best_data, train_y)

        preds_tmp = forest_model.predict(teste_X[selector.get_feature_names_out()])

        hash_table[value] = [accuracy_score(test_y, preds_tmp), f1_score(test_y, preds_tmp, average='weighted')]
        #print('Para as ', value, 'primeiras colunas')
        #print(accuracy_score(test_y, preds_tmp))
        #print(f1_score(test_y, preds_tmp, average='weighted'))
        #print()
        #print('-'*30)
    return hash_table

#no_const = func()
no_const = None

In [46]:
hash_table = {}
columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
for value in columns2select:
    selector = SelectKBest(f_classif, k=value)
    k_best_data = pd.DataFrame(selector.fit_transform(train_X, train_y), columns=selector.get_feature_names_out())
    #k_best_data

    forest_model = RandomForestClassifier(random_state=3)

    forest_model.fit(k_best_data, train_y)

    preds_tmp = forest_model.predict(teste_X[selector.get_feature_names_out()])

    hash_table[value] = [accuracy_score(test_y, preds_tmp), f1_score(test_y, preds_tmp, average='weighted')]
    #print('Para as ', value, 'primeiras colunas')
    #print(accuracy_score(test_y, preds_tmp))
    #print(f1_score(test_y, preds_tmp, average='weighted'))
    #print()
    #print('-'*30)

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 18  34  54  66  68  70  73  82 130 131 138 140 142 144 150 155 156 163
 193 203 263 273] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 18  34  54  66  68  70  73  82 130 131 138 140 142 144 150 155 156 163
 193 203 263 273] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/home/kalai/

In [52]:
for value in hash_table:
    print('Para as ',value, 'colunas selecionadas pelo cluster')
    print('acc: ', hash_table[value][0])
    print('f1: ', hash_table[value][1])
    print()

Para as  20 colunas selecionadas pelo cluster
acc:  0.6888888888888889
f1:  0.5958597883597883

Para as  30 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.6334259259259258

Para as  40 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.6380406212664277

Para as  50 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7077248677248676

Para as  60 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.711776522284997

Para as  100 colunas selecionadas pelo cluster
acc:  0.7777777777777778
f1:  0.7478781244298487

Para as  150 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7057507155867812

Para as  200 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.642057420418076

Para as  250 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7142267325873883



In [66]:


def removing_constants():
    hash_table = {}
    columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
    teste = df.drop(df.loc[:, (df == 0).all()].columns, axis=1)
    teste = teste.drop(['DII_Existence_ragged_P_wave', 'AVR_5', 'AVL_9', 'Amp_AVR_6'], axis=1)
    
    train, teste = train_test_split(teste, test_size=0.1, random_state=42)

    train_X = train.drop("Class", axis=1)
    train_y = train.Class

    teste_X = teste.drop("Class", axis=1)
    test_y = teste.Class
    
    for value in columns2select:
        selector = SelectKBest(f_classif, k=value)
        k_best_data = pd.DataFrame(selector.fit_transform(train_X, train_y), columns=selector.get_feature_names_out())

        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(k_best_data, train_y)

        preds_tmp = forest_model.predict(teste_X[selector.get_feature_names_out()])

        hash_table[value] = [accuracy_score(test_y, preds_tmp), f1_score(test_y, preds_tmp, average='weighted')]
    for value in hash_table:
        print('Para as ',value, 'colunas selecionadas pelo cluster')
        print('acc: ', hash_table[value][0])
        print('f1: ', hash_table[value][1])
        print()
    
    
removing_constants()

Para as  20 colunas selecionadas pelo cluster
acc:  0.6888888888888889
f1:  0.5958597883597883

Para as  30 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.6334259259259258

Para as  40 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.6380406212664277

Para as  50 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7077248677248676

Para as  60 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.711776522284997

Para as  100 colunas selecionadas pelo cluster
acc:  0.7777777777777778
f1:  0.7478781244298487

Para as  150 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7057507155867812

Para as  200 colunas selecionadas pelo cluster
acc:  0.7111111111111111
f1:  0.642057420418076

Para as  250 colunas selecionadas pelo cluster
acc:  0.7555555555555555
f1:  0.7142267325873883



In [68]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

# Separar os dados em features (X) e target (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Calcular o ganho de informação entre cada variável e a variável alvo
gain = mutual_info_classif(X, y)

# Criar um DataFrame com as variáveis e seus ganhos de informação
gain_df = pd.DataFrame({'Variable': X.columns, 'Gain': gain})

# Ordenar as variáveis por ganho de informação
gain_df = gain_df.sort_values(by='Gain', ascending=False)

# Exibir as variáveis em ordem decrescente de ganho de informação
gain_df

,Variable,Gain
13,Heart_rate,0.266902
112,V3_3,0.262045
195,Amp_AVR_8,0.231657
226,Amp_V1_9,0.220279
100,V2_3,0.215339
...,...,...
155,V6_10,0.000000
154,V6_9,0.000000
153,V6_8,0.000000
152,V6_7,0.000000


In [89]:
df.head()

,Age,Sex,Height,Weight,QRS_duration,P-R,Q-T,T_interval,P_interval,QRS,T,P,QRST,Heart_rate,Q_wave,R_wave,S_wave,R_lin_wave,S_lin_wave,Number_itrinsic_deflections,Existence_ragged_R_wave,Existence_diphasic_derivation_R_wave,Existence_ragged_P_wave,Existence_diphasic_drivation_P_wave,Existence_ragged_T_wave,Existence_diphasic_derivation_T_wave,DII_Q_wave,DII_R_wave,DII_S_wave,DII_R_lin_wave,DII_S_lin__wave,DII_Number_itrinsic_deflections,DII_Existence_ragged_R_wave,DII_Existence_diphasic_derivation_R_wave,DII_Existence_ragged_P_wave,DII_Existence_diphasic_drivation_P_wave,DII_Existence_ragged_T_wave,DII_Existence_diphasic_derivation_T_wave,DIII_Q_wave,DIII_R_wave,DIII_S_wave,DIII_R_lin__wave,DIII_S_lin__wave,DIII_Number_itrinsic_deflections,DIII_Existence_ragged_R_wave,DIII_Existence_diphasic_derivation_R_wave,DIII_Existence_ragged_P_wave,DIII_Existence_diphasic_drivation_P_wave,DIII_Existence_ragged_T_wave,DIII_Existence_diphasic_derivation_T_wave,AVR_1,AVR_2,AVR_3,AVR_4,AVR_5,AVR_6,AVR_7,AVR_8,AVR_9,AVR_10,AVR_11,AVR_12,AVL_1,AVL_2,AVL_3,AVL_4,AVL_5,AVL_6,AVL_7,AVL_8,AVL_9,AVL_10,AVL_11,AVL_12,AVF_1,AVF_2,AVF_3,AVF_4,AVF_5,AVF_6,AVF_7,AVF_8,AVF_9,AVF_10,AVF_11,AVF_12,V1_1,V1_2,V1_3,V1_4,V1_5,V1_6,V1_7,V1_8,V1_9,V1_10,V1_11,V1_12,V2_1,V2_2,V2_3,V2_4,V2_5,V2_6,V2_7,V2_8,V2_9,V2_10,V2_11,V2_12,V3_1,V3_2,V3_3,V3_4,V3_5,V3_6,V3_7,V3_8,V3_9,V3_10,V3_11,V3_12,V4_1,V4_2,V4_3,V4_4,V4_5,V4_6,V4_7,V4_8,V4_9,V4_10,V4_11,V4_12,V5_1,V5_2,V5_3,V5_4,V5_5,V5_6,V5_7,V5_8,V5_9,V5_10,V5_11,V5_12,V6_1,V6_2,V6_3,V6_4,V6_5,V6_6,V6_7,V6_8,V6_9,V6_10,V6_11,V6_12,Amp_JJ_wave,Amp_Q_wave,Amp_R_wave,Amp_S_wave,Amp_R_lin_wave,Amp_S_lin_wave,Amp_P_wave,Amp_T_wave,QRSA,QRSTA,Amp_DII_1,Amp_DII_2,Amp_DII_3,Amp_DII_4,Amp_DII_5,Amp_DII_6,Amp_DII_7,Amp_DII_8,Amp_DII_9,Amp_DII_10,Amp_DIII_1,Amp_DIII_2,Amp_DIII_3,Amp_DIII_4,Amp_DIII_5,Amp_DIII_6,Amp_DIII_7,Amp_DIII_8,Amp_DIII_9,Amp_DIII_10,Amp_AVR_1,Amp_AVR_2,Amp_AVR_3,Amp_AVR_4,Amp_AVR_5,Amp_AVR_6,Amp_AVR_7,Amp_AVR_8,Amp_AVR_9,Amp_AVR_10,Amp_AVL_1,Amp_AVL_2,Amp_AVL_3,Amp_AVL_4,Amp_AVL_5,Amp_AVL_6,Amp_AVL_7,Amp_AVL_8,Amp_AVL_9,Amp_AVL_10,Amp_AVF_1,Amp_AVF_2,Amp_AVF_3,Amp_AVF_4,Amp_AVF_5,Amp_AVF_6,Amp_AVF_7,Amp_AVF_8,Amp_AVF_9,Amp_AVF_10,Amp_V1_1,Amp_V1_2,Amp_V1_3,Amp_V1_4,Amp_V1_5,Amp_V1_6,Amp_V1_7,Amp_V1_8,Amp_V1_9,Amp_V1_10,Amp_V2_1,Amp_V2_2,Amp_V2_3,Amp_V2_4,Amp_V2_5,Amp_V2_6,Amp_V2_7,Amp_V2_8,Amp_V2_9,Amp_V2_10,Amp_V3_1,Amp_V3_2,Amp_V3_3,Amp_V3_4,Amp_V3_5,Amp_V3_6,Amp_V3_7,Amp_V3_8,Amp_V3_9,Amp_V3_10,Amp_V4_1,Amp_V4_2,Amp_V4_3,Amp_V4_4,Amp_V4_5,Amp_V4_6,Amp_V4_7,Amp_V4_8,Amp_V4_9,Amp_V4_10,Amp_V5_1,Amp_V5_2,Amp_V5_3,Amp_V5_4,Amp_V5_5,Amp_V5_6,Amp_V5_7,Amp_V5_8,Amp_V5_9,Amp_V5_10,Amp_V6_1,Amp_V6_2,Amp_V6_3,Amp_V6_4,Amp_V6_5,Amp_V6_6,Amp_V6_7,Amp_V6_8,Amp_V6_9,Amp_V6_10,Class
0,75.0,1,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,13.0,64.0,-2.0,63.0,0.0,52.0,44.0,0.0,0.0,32.0,0,0,0,0,0,0,0.0,44.0,20.0,36.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,40.0,0.0,0.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,36.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,32.0,0.0,0.0,0.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,52.0,0.0,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,48.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,52.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,48.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56.0,44.0,0.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.2,0.0,6.1,-1.0,0.0,0.0,0.6,2.1,13.6,30.8,0.0,0.0,1.7,-1.0,0.6,0.0,1.3,1.5,3.7,14.5,0.1,-5.2,1.4,0.0,0.0,0.0,0.8,-0.6,-10.7,-15.6,0.4,-3.9,0.0,0.0,0.0,0.0,-0.8,-1.7,-10.1,-22.0,0.0,0.0,5.7,-1.0,0.0,0.0,-0.1,1.2,14.1,22.5,0.0,-2.5,0.8,0.0,0.0,0.0,1.0,0.4,-4.8,-2.7,0.1,-6.0,0.0,0.0,0.0,0.0,-0.8,-0.6,-24.0,-29.7,0.0,0.0,2.0,-6.4,0.0,0.0,0.2,2.9,-12.6,15.2,-0.1,0.0,8.4,-10.0,0.0,0.0,0.6,5.9,-3.9,52.7,-0.3,0.0,15.2,-8.4,0.0,0.0,0.9,5.1,17.7,70.7,-0.4,0.0,13.5,-4.0,0.0,0.0,0.9,3.9,25.5,62.9,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8
1,56.0,0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0

In [90]:
n=3
df.iloc[:,:n]

,Age,Sex,Height
0,75.0,1,190.0
1,56.0,0,165.0
2,54.0,1,172.0
3,55.0,1,175.0
5,13.0,1,169.0
...,...,...,...
447,53.0,0,160.0
448,37.0,1,190.0
449,36.0,1,166.0
450,32.0,0,155.0


In [97]:
def mutual_info():
    hash_table = {}   
    columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
    
    bigger_then_zero=gain_df[gain_df['Gain']>0]
    selected_columns = df[bigger_then_zero['Variable']] 
    selected_columns['Class'] = df['Class']
    #display(bigger_then_zero)
    #display(testando)
    
    
    train, teste = train_test_split(selected_columns, test_size=0.1, random_state=42)
    
    train_X = train.drop("Class", axis=1)
    train_y = train.Class
    

    teste_X = teste.drop("Class", axis=1)
    #test_y = teste.Class
    
    for value in columns2select:

        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(train_X.iloc[:,:value], train_y)
        
        preds_tmp = forest_model.predict(teste_X.iloc[:,:value])

        hash_table[value] = [accuracy_score(test_y, preds_tmp), f1_score(test_y, preds_tmp, average='weighted')]
    for value in hash_table:
        print('Para as ',value, 'colunas selecionadas pelo mutual gain')
        print('acc: ', hash_table[value][0])
        print('f1: ', hash_table[value][1])
        print()
    
    
mutual_info()

/tmp/ipykernel_10440/1152734646.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['Class'] = df['Class']


Para as  20 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.7125172630257377

Para as  30 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.7154802259887006

Para as  40 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6696296296296296

Para as  50 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6696296296296296

Para as  60 colunas selecionadas pelo mutual gain
acc:  0.7777777777777778
f1:  0.7429934535019282

Para as  100 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.711776522284997

Para as  150 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6833272616879174

Para as  200 colunas selecionadas pelo mutual gain
acc:  0.7111111111111111
f1:  0.6431156214762772

Para as  250 colunas selecionadas pelo mutual gain
acc:  0.7777777777777778
f1:  0.7429934535019282



In [99]:
import pandas as pd
from sklearn.feature_selection import mutual_info_regression

# Separar os dados em features (X) e target (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Calcular a informação mútua entre cada variável e a variável alvo
mutual_info = mutual_info_regression(X, y)

# Criar um DataFrame com as variáveis e suas informações mútuas
mutual_info_df = pd.DataFrame({'Variable': X.columns, 'Mutual Information': mutual_info})

# Ordenar as variáveis por informação mútua
mutual_info_df = mutual_info_df.sort_values(by='Mutual Information', ascending=False)

# Exibir as variáveis em ordem decrescente de informação mútua
mutual_info_df

,Variable,Mutual Information
195,Amp_AVR_8,0.248828
112,V3_3,0.225043
100,V2_3,0.205002
222,Amp_V1_5,0.202917
13,Heart_rate,0.199872
...,...,...
117,V3_8,0.000000
65,AVL_4,0.000000
216,Amp_AVF_9,0.000000
66,AVL_5,0.000000


In [101]:
def mutual_regression():
    hash_table = {}   
    columns2select = [20, 30, 40, 50, 60, 100, 150, 200, 250]
    
    bigger_then_zero=mutual_info_df[mutual_info_df['Mutual Information']>0]
    selected_columns = df[bigger_then_zero['Variable']] 
    selected_columns['Class'] = df['Class'].
    #display(bigger_then_zero)
    #display(testando)
    
    
    train, teste = train_test_split(selected_columns, test_size=0.1, random_state=42)
    
    train_X = train.drop("Class", axis=1)
    train_y = train.Class
    

    teste_X = teste.drop("Class", axis=1)
    #test_y = teste.Class
    
    for value in columns2select:

        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(train_X.iloc[:,:value], train_y)
        
        preds_tmp = forest_model.predict(teste_X.iloc[:,:value])

        hash_table[value] = [accuracy_score(test_y, preds_tmp), f1_score(test_y, preds_tmp, average='weighted')]
    for value in hash_table:
        print('Para as ',value, 'colunas selecionadas pelo mutual gain')
        print('acc: ', hash_table[value][0])
        print('f1: ', hash_table[value][1])
        print()
    
    
mutual_regression()

/tmp/ipykernel_10440/3346210280.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns['Class'] = df['Class'].loc[:]


Para as  20 colunas selecionadas pelo mutual gain
acc:  0.7111111111111111
f1:  0.6334259259259258

Para as  30 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.7123056228140976

Para as  40 colunas selecionadas pelo mutual gain
acc:  0.7777777777777778
f1:  0.7525536062378169

Para as  50 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6764495530012772

Para as  60 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.7098412698412697

Para as  100 colunas selecionadas pelo mutual gain
acc:  0.7555555555555555
f1:  0.7077248677248676

Para as  150 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6886772486772486

Para as  200 colunas selecionadas pelo mutual gain
acc:  0.7333333333333333
f1:  0.6886772486772486

Para as  250 colunas selecionadas pelo mutual gain
acc:  0.7777777777777778
f1:  0.7478781244298487



In [104]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
# Separar os dados em features (X) e target (y)
X = df.drop('Class', axis=1)
y = df['Class']

# Criar o seletor de features sequenciais usando regressão linear
selector = SequentialFeatureSelector(LinearRegression(), direction='forward', n_features_to_select = 6)

# Aplicar o seletor às features (X) e target (y)
#X_selected = selector.fit_transform(X, y)

sfs = pd.DataFrame(selector.fit_transform(X, y), columns=selector.get_feature_names_out())
sfs

,QRS_duration,AVL_6,V1_4,V1_5,V1_8,Amp_R_wave
0,91.0,32.0,0.0,0.0,0.0,6.1
1,81.0,24.0,0.0,0.0,0.0,7.2
2,138.0,24.0,76.0,0.0,0.0,4.5
3,100.0,44.0,0.0,0.0,0.0,7.8
4,100.0,16.0,0.0,0.0,0.0,2.7
...,...,...,...,...,...,...
443,80.0,28.0,0.0,0.0,0.0,6.2
444,100.0,0.0,0.0,0.0,0.0,3.3
445,108.0,48.0,44.0,28.0,0.0,8.4
446,93.0,12.0,0.0,0.0,0.0,5.2


In [108]:
def sequential_feature():
    hash_table = {}   
    columns2select = [6, 12, 20, 30, 40, 50, 60, 100, 150, 200, 250]
    
    train, teste = train_test_split(df, test_size=0.1, random_state=42)
    
    train_X = train.drop("Class", axis=1)
    train_y = train.Class
    
    teste_X = teste.drop("Class", axis=1)
    teste_y = teste.Class
    
    for n_features in columns2select:
        X = df.drop('Class', axis=1)
        y = df['Class']

        # Criar o seletor de features sequenciais usando regressão linear
        selector = SequentialFeatureSelector(LinearRegression(), direction='forward', n_features_to_select = n_features)

        sfs = pd.DataFrame(selector.fit_transform(train_X, train_y), columns=selector.get_feature_names_out())
        
        
        forest_model = RandomForestClassifier(random_state=3)

        forest_model.fit(sfs, train_y)
        
        preds_tmp = forest_model.predict(teste_X[list(sfs.columns)])

        hash_table[n_features] = [accuracy_score(teste_y, preds_tmp), f1_score(teste_y, preds_tmp, average='weighted')]
        for value in hash_table:
            print('Para as ',value, 'colunas selecionadas pelo mutual gain')
            print('acc: ', hash_table[value][0])
            print('f1: ', hash_table[value][1])
            print()


sequential_feature()

Para as  6 colunas selecionadas pelo mutual gain
acc:  0.6
f1:  0.506031746031746

Para as  6 colunas selecionadas pelo mutual gain
acc:  0.6
f1:  0.506031746031746

Para as  12 colunas selecionadas pelo mutual gain
acc:  0.6222222222222222
f1:  0.5371428571428573

Para as  6 colunas selecionadas pelo mutual gain
acc:  0.6
f1:  0.506031746031746

Para as  12 colunas selecionadas pelo mutual gain
acc:  0.6222222222222222
f1:  0.5371428571428573

Para as  20 colunas selecionadas pelo mutual gain
acc:  0.6444444444444445
f1:  0.5504273504273504

Para as  6 colunas selecionadas pelo mutual gain
acc:  0.6
f1:  0.506031746031746

Para as  12 colunas selecionadas pelo mutual gain
acc:  0.6222222222222222
f1:  0.5371428571428573

Para as  20 colunas selecionadas pelo mutual gain
acc:  0.6444444444444445
f1:  0.5504273504273504

Para as  30 colunas selecionadas pelo mutual gain
acc:  0.6666666666666666
f1:  0.5587301587301587

Para as  6 colunas selecionadas pelo mutual gain
acc:  0.6
f1:  0.50

In [29]:
"""
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestRegressor
# Separar os dados em features (X) e target (y)
train, teste = train_test_split(df, test_size=0.1, random_state=42)
    
train_X = train.drop("Class", axis=1)
train_y = train.Class

teste_X = teste.drop("Class", axis=1)
teste_y = teste.Class

knn = KNeighborsClassifier(n_neighbors=3)

selector = SequentialFeatureSelector(knn)

# Aplicar o seletor às features (X) e target (y)
#X_selected = selector.fit_transform(X, y)

sfs = pd.DataFrame(selector.fit_transform(train_X, train_y), columns=selector.get_feature_names_out())
sfs
"""

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

...

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/home/kalai/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.w

KeyboardInterrupt: 

In [28]:
forest_model = RandomForestClassifier(random_state=3)
atribute_columns = sfs.columns
forest_model.fit(sfs, train_y)

preds_tmp = forest_model.predict(teste_X[atribute_columns])


print(accuracy_score(test_y, preds_tmp))
print(f1_score(test_y, preds_tmp, average='weighted'))
print()

0.6222222222222222
0.626031746031746

